# Basic EO Operations

In [1]:
import os
os.environ['HTTP_PROXY']=""
os.environ['HTTPS_PROXY']=""

In [2]:
import os
import rasterio as rio
from rasterio import windows
from rasterio.plot import show
from rasterio.features import geometry_mask, rasterize
import glob
from itertools import product
from matplotlib import pyplot
from shapely.geometry import Polygon
import osmnx as ox
import matplotlib.pyplot as plt
from functools import partial
import pyproj
from shapely.ops import transform

In [3]:
def tiles_image(raster, width, height):
    nols, nrows = raster.meta['width'], raster.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform_tile = windows.transform(window, raster.transform)
        yield window, transform_tile

# Si no esta en 4326 convertimos porque osmnx solo acepta lat, lon        
def get_building_osm(geom, outfile_bf):
    if tile.crs != "4326":     
        project = partial(pyproj.transform,pyproj.Proj(init='epsg:32611'), pyproj.Proj(init='epsg:4326'))
        geom_tile = transform(project, geom)
        print (geom_tile)
        bfprints= ox.footprints.footprints_from_polygon(geom_tile, footprint_type ='building')        
        if not bfprints.empty:
            bfprints.drop(['nodes'], axis = 1, inplace=True)
            bfprints.to_crs({'init': 'epsg:32611'}, inplace=True)
            bfprints.to_file(outfile_bf, driver='GeoJSON')
    else: 
        bfprints= ox.footprints.footprints_from_polygon(geom, footprint_type ='building')
        if not bfprints.empty:
            bfprints.drop(['nodes'], axis = 1, inplace=True)
            bfprints.to_file(outfile_bf, driver='GeoJSON')
    return 1

In [ ]:
out_path = r"E://Datasets//SN//Train//IMG//"
out_path_b = r"E://Datasets//SN//Train//Buildings"
tif_ext="*.tif"
output_filename = 'Tile{}.tif'
output_filename_b = 'Buildings{}.geojson'
tile_width, tile_height = 3000, 3000

if not os.path.exists(out_path):
    os.makedirs(out_path)

if not os.path.exists(out_path_b):
    os.makedirs(out_path_b)

filepath = 'E://Datasets//SN//CapeTown_012545731010_0//012545731010_01_003//012545731010_01//012545731010_01_P001_PSH//20MAY14085945-S3DS_R1C2-012545731010_01_P001.TIF'
flag=0
print('Read Data to Tile')
with rio.open(filepath) as img:
    meta = img.meta.copy()
    for window, transform_tile in tiles_image(img, tile_width, tile_height):        
        #show(img.read(window=window))
        meta['transform'] = transform_tile
        meta['width'], meta['height'] = window.width, window.height
        outfile = os.path.join(out_path,output_filename.format(int(flag)))
        outfile_bf = os.path.join(out_path_b,output_filename_b.format(int(flag)))        
        with rio.open(outfile, 'w', **meta) as tile:
            poly = Polygon([(tile.bounds.left, tile.bounds.top), (tile.bounds.left, tile.bounds.bottom), (tile.bounds.right, tile.bounds.bottom), (tile.bounds.right, tile.bounds.top)])       
            get_building_osm(poly, outfile_bf)
            tile.write(img.read(window=window))    
        print('File'+outfile+' created')
        flag=flag+1

Read Data to Tile
POLYGON ((-120.6637832864187 56.2974761465626, -120.6628469632813 56.28769984366632, -120.6452689119959 56.28821919500452, -120.6462007660517 56.29799568879134, -120.6637832864187 56.2974761465626))
FileE://Datasets//SN//Train//IMG//Tile0.tif created
POLYGON ((-120.6628469632813 56.28769984366632, -120.6619112244567 56.27792351501332, -120.6443376394826 56.278442675558, -120.6452689119959 56.28821919500452, -120.6628469632813 56.28769984366632))
FileE://Datasets//SN//Train//IMG//Tile1.tif created
POLYGON ((-120.6619112244567 56.27792351501332, -120.6609760694443 56.26814716061028, -120.6434069480137 56.26866613045836, -120.6443376394826 56.278442675558, -120.6619112244567 56.27792351501332))
FileE://Datasets//SN//Train//IMG//Tile2.tif created
POLYGON ((-120.6609760694443 56.26814716061028, -120.6600414977441 56.25837078046391, -120.6424768370913 56.25888955971225, -120.6434069480137 56.26866613045836, -120.6609760694443 56.26814716061028))
FileE://Datasets//SN//Train/